In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
words = open('names.txt').read().splitlines()


In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [183]:
# build the dataset
import random

blocksize = 3

def build_dataset(words):
    X = []
    Y = []
    for word in words:
        context = [0] * blocksize
        for ch in word + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
[X, Y] = build_dataset(words[:n1])
[XV, YV] = build_dataset(words[n1:n2])
[XT, YT] = build_dataset(words[n2:])

In [184]:
# Two dimensional embedding
C = torch.randn((27, 10), requires_grad=True)
embedding_sum_size = C.shape[1] * blocksize

In [185]:
W1 = torch.randn((embedding_sum_size, 200), requires_grad=True)
b1 = torch.randn(W1.shape[1], requires_grad=True)
W2 = torch.randn((W1.shape[1], 27), requires_grad=True)
b2 = torch.randn(W2.shape[1], requires_grad=True)
parameters = [C, W1, b1, W2, b2]

In [194]:
def forward(emb, target):
    h = torch.tanh(emb.view(-1, embedding_sum_size) @ W1 + b1)
    # Output layer
    logits = h @ W2 + b2
    
    # Cross entropy loss
    anll = F.cross_entropy(logits, target)

    return anll

for _ in range(10000):
    ix = torch.randint(0, X.shape[0], (64, ))
    # Forward pass
    # Embedded values
    emb = C[X[ix]]
    
    # Cross entropy loss
    anll = forward(emb, Y[ix])

    for p in parameters:
        p.grad = None
        
    anll.backward()
    
    learning_rate = 0.05
    for p in parameters:
        p.data += -learning_rate * p.grad
        
print(f'Train: {forward(C[X], Y)} Validation: {forward(C[XV], YV)} Test: {forward(C[XT], YT)}')

Train: 2.316354513168335 Validation: 2.331671953201294 Test: 2.3411452770233154


In [187]:
# training split (for training), # dev/validation (for hyperparemeter tuning), # test split (for final evaluation)

In [195]:
for _ in range(20):
    out = []
    context = [0] * blocksize
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

zeah.
buslaeseseuaushur.
raymeelyanah.
adie.
mulem.
issimarlandrostae.
malon.
mashet.
matir.
nana.
ala.
sily.
jona.
jyon.
kasharri.
tro.
zoler.
scee.
jasha.
gunta.
